This notebook is specific to the [Waifu Diffusion v1.4](https://huggingface.co/hakurei/waifu-diffusion-v1-4) model only. Click here how to prompts this [model](https://gist.github.com/harubaru/8581e780a1cf61352a739f2ec2eef09b). Want to use another model? check [here](https://github.com/Lycantant/invoke-ai-gui-colab). <br>

Not familiar with the prompt technique used in InvokeAi? Read [here](https://invoke-ai.github.io/InvokeAI/features/PROMPTS/). And also each model has a different prompt technique. So you should read the links above carefully before starting, so your time will not be wasted.

---
## Requirements
1. Login to ngrok [here](https://dashboard.ngrok.com/login), and create authtoken [here](https://dashboard.ngrok.com/get-started/your-authtoken).
2. Requires 5.5 Gb disk space on a [Google Drive](https://drive.google.com/drive/my-drive)
3. Don't rush, launch code in each form below one by one.
---
####Visit my Github repo [here](https://github.com/Lycantant/invoke-ai-gui-colab/).
Give a ⭐ if you like it 😁 <br>
Semoga beruntung 🙂

In [ ]:
# @title 01. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 02. During this stage, Colab crashes, it's normal
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title 03. Installing [InvokeAi](https://github.com/invoke-ai/InvokeAI) { display-mode: "form" }
%cd /home

!git clone --depth 1 -n https://github.com/invoke-ai/InvokeAI.git
%cd InvokeAI
!git checkout 11601d4378e1903bdbefd1b918205891864743c4 # v2.2.5p2

%cd ..

!git clone https://github.com/Lycantant/invoke-ai-gui-colab.git

%cd invoke-ai-gui-colab

print('\x1b[1;32m')
%cp -v cross_attention_control.py /home/InvokeAI/ldm/models/diffusion/cross_attention_control.py
%cp -v globals.py /home/InvokeAI/ldm/invoke/globals.py
print('\x1b[0m')

print('\x1b[1;32m')
%cd ../InvokeAI
!ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml
!ls -la
print('\x1b[0m')

In [ ]:
#@title 04. Installation normally takes ~10 minutes { display-mode: "form" }
!pip install pyngrok --quiet
!conda env update
!source activate invokeai ; python scripts/configure_invokeai.py --skip-sd-weights --yes

%cp ../invoke-ai-gui-colab/models_yaml/models_waifu_diffusion_v1_4.yaml /root/invokeai/configs/models.yaml

In [ ]:
#@title 05. Downloading Waifu Diffusion v1.4 model (requires 5.5 Gb disk space on a Google Drive){ display-mode: "form" }
import os

os.system('cd /')
os.system('mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1')

# Waifu Diffusion 1.4
if os.path.exists('/content/drive/MyDrive/models/wd-1-4-anime_e1.ckpt'):
  print('Model wd-1-4-anime_e1.ckpt already exists')
else:
  print('Downloading wd-1-4-anime_e1.ckpt')
  os.system('wget -O /content/drive/MyDrive/models/wd-1-4-anime_e1.ckpt https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt')

# vae Waifu Diffusion 1.4
if os.path.exists('/content/drive/MyDrive/models/kl-f8-anime2.ckpt'):
  print('Model kl-f8-anime2.ckpt already exists')
else:
  print('Downloading kl-f8-anime2.ckpt')
  os.system('wget -O /content/drive/MyDrive/models/kl-f8-anime2.ckpt https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt')

# Waifu Diffusion 1.4
!ln -s  /content/drive/MyDrive/models/wd-1-4-anime_e1.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/wd-1-4-anime_e1.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/wd-1-4-anime_e1.ckpt

# vae Waifu Diffusion 1.4
!ln -s  /content/drive/MyDrive/models/kl-f8-anime2.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/kl-f8-anime2.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/kl-f8-anime2.ckpt

# embeddings
!ln -s /home/invoke-ai-gui-colab/embeddings/*.pt /root/invokeai/embeddings/
!ls -l /root/invokeai/embeddings/

In [ ]:
#@title 06. Running Invoke AI service { display-mode: "both" }
#@markdown Go to [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) , and paste your authtoken here.

ngrok_token = "" #@param {type:"string"}
nsfw_checker = 0 #@param {type:"slider", min:0, max:1, step:1}
used_vae = "kl-f8-anime2" #@param ["kl-f8-anime2"]

import os
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9090).public_url
print(f'Invoke Ai public url: {public_url}')

%cd /home/InvokeAI

model_name = "waifu_diffusion_v1_4"
if used_vae == "kl-f8-anime2":
    model_name = "waifu_diffusion_v1_4"

if nsfw_checker:
    !source activate invokeai ; python scripts/invoke.py --web --outdir "/content/drive/MyDrive/outputs" --model $model_name
else:
    !source activate invokeai ; python scripts/invoke.py --web --no-nsfw_checker --outdir "/content/drive/MyDrive/outputs" --model $model_name

#### Wait until the local url (`http://localhost:9090`) is available.
#### Then click/open public url, it will look something like this ➡ `http://3877-35-204-151-175.ngrok.io`
#### Refresh the page if you see a blank screen
---
Semoga beruntung 😀